In [2]:
## Create JSON that maps macaque vertex to macaque parcellation, human parcellation and vertex
## Load in everything required
import nibabel as nib
import numpy as np
import json
LMarkov = nib.load("../../../Atlas/alignment_macaque-human/cross-species_parcellation/L.Markov.monkey.10k_fs_LR.label.gii").agg_data()
LAparc = nib.load("../../../Atlas/aparc/fs_LR.aparc.L.10k.func.gii").agg_data()
LMarkovHuman = nib.load("../../../Atlas/alignment_macaque-human/cross-species_parcellation/L.Markov.monkey-to-human.10k_fs_LR.label.gii").agg_data()
LAparcMonkey = nib.load("../../../Atlas/alignment_macaque-human/cross-species_parcellation/L.Aparc.human-to-monkey.10k_fs_LR.func.gii").agg_data()
aparcJSON = "../../../data_v2.0/demographics/aparc_order.json"
markovJSON = "../../../data_v2.0/demographics/markov_order.json"
with open(aparcJSON, 'r') as file:
    aparcOrder = json.load(file)

with open(markovJSON, 'r') as file:
    markovOrder = json.load(file)
aparcOrder['unknown'] = 0
markovOrder['unknown'] = 0

In [3]:
## Create markov JSON
monkey_to_human_dict_list = []
for index, label in enumerate(LMarkov):
    dict = {}
    dict['MarkovLabel'] = int(label)
    dict['MarkovName'] = list(markovOrder.keys())[list(markovOrder.values()).index(label)]
    dict['AparcLabel'] = int(LAparcMonkey[index])
    dict['AparcName'] = list(aparcOrder.keys())[list(aparcOrder.values()).index(LAparcMonkey[index])]
    monkey_to_human_dict_list.append(dict)

human_to_monkey_dict_list = []
for index, label in enumerate(LAparc):
    dict = {}
    dict['AparcLabel'] = int(label)
    dict['AparcName'] = list(aparcOrder.keys())[list(aparcOrder.values()).index(label)]
    dict['MarkovLabel'] = int(LMarkovHuman[index])
    dict['MarkovName'] = list(markovOrder.keys())[list(markovOrder.values()).index(LMarkovHuman[index])]
    human_to_monkey_dict_list.append(dict)

with open('monkey_to_human_mapping.json', 'w') as f:
    json.dump(monkey_to_human_dict_list, f, indent=2)
with open('human_to_monkey_mapping.json', 'w') as f:
    json.dump(human_to_monkey_dict_list, f, indent=2)

In [14]:
## Find which unique labels in markov-human map to which aparc labels
markovToAparcMapping = np.zeros(len(np.unique(LMarkovHuman)))
for index, label in enumerate(np.unique(LMarkovHuman)):
    indicesForLabel = np.where(LMarkovHuman == label)
    aparcArray = LAparc[indicesForLabel]
    counts = np.bincount(aparcArray)
    mostCommon = np.argmax(counts)
    markovToAparcMapping[index] = mostCommon

In [15]:
markovToAparcMapping

array([ 0., 22., 22., 22., 22., 28., 14., 12., 12., 12., 14., 25., 26.,
       17., 14., 18.,  2.,  2., 28., 28., 23., 20., 20., 27., 27., 31.,
       31., 17., 31., 27., 27., 27., 27., 27., 20., 31., 30., 29.,  0.,
       24., 28., 28., 24., 18., 28., 27., 15., 20., 35., 15., 30., 29.,
       30., 29., 31.,  8., 14., 12., 30., 30., 30.,  0., 15., 29.,  0.,
       20., 10., 22., 31., 30., 15.,  0.,  6.,  9.,  9.,  6.,  0.,  7.,
       11.,  7., 16., 16., 31., 21., 25., 13., 29., 11.,  8., 25., 25.,
       29.])